In [38]:
%load_ext lab_black
%matplotlib inline

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymc3 as pm
import scipy as sp
import seaborn as sns
import theano.tensor as tt
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

sns.set(context="notebook", font_scale=1.2, rc={"figure.figsize": (12, 5)})
plt.style.use(["seaborn-colorblind", "seaborn-darkgrid"])

RANDOM_SEED = 8927
np.random.seed(286)

PREDICTORS = [
    "climat_affaires",
    "conf_menages",
    "prix_gazole",
    "inflation",
    "pib",
    "net_app",
    "chomage",
]
PARTIES = ["farleft", "left", "green", "center", "right", "farright", "other"]
PARTIES_INT = [f"{p}_int" for p in PARTIES]

# Helper function
def stdz(series: pd.Series):
    """Standardize the given pandas Series"""
    return (series - series.mean()) / series.std()

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [32]:
d = pd.read_json("full_funds.json")

for p in PREDICTORS:
    d[f"{p}_std"] = stdz(d[p])
d[PARTIES_INT] = (d[PARTIES] * 100).astype(int)
d.head()

,departement,date,climat_affaires,conf_menages,prix_gazole,inflation,pib,net_app,chomage,election,farleft,left,green,center,right,farright,other
0,ain,1992-03-22,95.5,102,0.54,3.2,0.546012,-20.48,5.0,dep1992,4.82,13.89,7.41,24.88,20.64,12.59,15.77
1,ain,1993-03-21,76.7,99,0.54,2.4,-0.213516,-28.59,6.3,leg1993,6.47,11.82,5.53,25.60,22.74,14.83,13.01
2,ain,1994-03-20,93.5,100,0.60,2.1,0.225626,-10.54,7.4,dep1994,6.98,12.16,5.48,26.17,14.76,11.49,22.96
3,ain,1994-06-12,103.0,104,0.60,2.1,0.658377,-8.29,7.6,euro1994,6.75,13.39,5.52,25.48,26.56,12.11,10.19
4,ain,1995-04-23,107.7,107,0.59,1.6,0.480469,-5.03,6.9,pres1995,11.04,20.50,3.91,19.80,19.23,19.86,5.66


In [34]:
d[
    [
        "climat_affaires_std",
        "conf_menages_std",
        "prix_gazole_std",
        "inflation_std",
        "pib_std",
        "net_app_std",
        "chomage_std",
    ]
].describe().round(2)

,climat_affaires_std,conf_menages_std,prix_gazole_std,inflation_std,pib_std,net_app_std,chomage_std
count,1889.00,1889.00,1889.00,1889.00,1889.00,1889.00,1889.00
mean,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.61,-1.74,-1.23,-2.14,-3.59,-2.36,-2.43
25%,-0.52,-0.33,-0.74,-0.26,-0.27,-0.66,-0.71
50%,0.24,-0.11,-0.26,0.01,0.23,-0.08,-0.07
75%,0.68,0.65,0.74,0.55,0.66,0.54,0.52
max,1.28,2.39,2.19,2.02,1.15,1.92,3.42


In [42]:
dpt_id, dpts = d.departement.factorize(sort=True)
Ndpts = len(dpts)
N = d[PARTIES_INT].sum(axis=1).values
R_obs = d[PARTIES_INT].values

with pm.Model() as m_itcpt:
    a = pm.Normal('a', 0., 10.)
    sigma_dpt = pm.Exponential('sigma_dpt', 1.)
    
    a_dpt = pm.Normal('a_dpt', a, sigma_dpt, shape=Ndpts)
    
    p = tt.nnet.softmax(a_dpt[dpt_id])
    R = pm.Multinomial("R", n=N, p=p, observed=R_obs)
    
    prior_checks = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
    #trace_itcpt = pm.sample(1000, tune=2000, cores=2, random_seed=RANDOM_SEED)

/anaconda/envs/fund/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  if sys.path[0] == '':


ValueError: Input dimension mis-match. (input[0].shape[1] = 7, input[1].shape[1] = 1889)